# Laboratorio de Control Térmico - TempLABUdeA
## Modelado del Sistema - Fase 1

In [1]:

# Parámetros identificados
Kp = 0.3593          # Ganancia del proceso
tm = 10.3349         # Tiempo muerto (s)
tau = 171.7428       # Constante de tiempo (s)

print(f"Modelo FOPDT identificado:")
print(f"G(s) = {Kp} * exp(-{tm} * s) / ({tau} * s + 1)")

Modelo FOPDT identificado:
G(s) = 0.3593 * exp(-10.3349 * s) / (171.7428 * s + 1)


## Sintonía P-only (ITAE)

In [2]:
import sympy as sp

s = sp.symbols('s')
Kp_val = 0.3593
tm_val = 10.3349
tau_val = 171.7428

# Modelo simbólico
G_s = Kp_val * sp.exp(-tm_val * s) / (tau_val * s + 1)
G_s.simplify()

0.3593*exp(-10.3349*s)/(171.7428*s + 1)


### Interpretación:
El modelo FOPDT describe el comportamiento dinámico de la temperatura frente a cambios en la potencia de calefacción con:
- Una **ganancia de proceso** $ K_p = 0.3593 $
- Un **tiempo muerto** $ \theta_p = 10.33\,s $
- Una **constante de tiempo** $\tau_p = 171.74\,s $

Este modelo será utilizado para diseñar el controlador PID en la siguiente fase.
## Diseño de PID mediante Ziegler-Nichols - Fase 2
- Encontrando parametros del PID
- $ K_C =(1.2 * \tau_p) / (K_p * \theta_p) $   
- $ Ti = 2 * \theta_p $
- $ Td = 0.5 * \theta_p $

In [6]:
Kc = (1.2 * tau_val) / (Kp_val * tm)
Ti = 2 * tm
Td = 0.5 * tm
print("Parámetros del controlador PID calculados:")
print(f"Kc = {Kc:.4f}")
print(f"Ti = {Ti:.4f} s")
print(f"Td = {Td:.4f} s")


Parámetros del controlador PID calculados:
Kc = 55.5004
Ti = 20.6698 s
Td = 5.1674 s


### Implementacion  de simulacion

![Simulacio.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Simulacio.png)
![Resultados.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Resultados.png)

- Periodo $P_u = 37.859s$
- $K_p= 104$

- $K_p =0,6K_u$
- $T_i = 0,5P_u$,
- $T_d = 0,125P_u$

In [8]:
Kp=104
Pu=37.859
Ku=Kp*0.6
Ti=0.5*Pu
Td=0.125*Pu
print("Parámetros del controlador PID calculados:") 
print(f"Kc = {Ku:.4f}")
print(f"Ti = {Ti:.4f} s")
print(f"Td = {Td:.4f} s")


Parámetros del controlador PID calculados:
Kc = 62.4000
Ti = 18.9295 s
Td = 4.7324 s


### Implementacion del sistema

$$ 
P=K_c

 $$
  $$
  I=K_c/T_i

 $$
  $$
  D=K_c*T_d
 $$

In [9]:
P=Kc
I=Kc/Ti
D=Kc*Td
print("Parámetros del controlador PID calculados:") 
print(f"Kp = {P:.4f}")
print(f"Ki = {I:.4f}")
print(f"Kd = {D:.4f}")

Parámetros del controlador PID calculados:
Kp = 55.5004
Ki = 2.9320
Kd = 262.6488


![Sistema.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Sistema.png)

### Resultado sistema estabilizado
![Resultado.png](https://raw.githubusercontent.com/jbcgames/Lab_5_Control/refs/heads/main/Resultado.png)

## Etapa 3 Implementeacion Practica

### Script Python

In [ ]:
"""
Prueba de escalón con registro de datos en Python
Adaptado y documentado para el Laboratorio de Control de Temperatura (TempLabUdeA)
Universidad de Antioquia - Sistemas de Control
"""
! pip install tclab numpy matplotlib pandas datetime
import tclab 
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime


def ejecutar_control(tipo_controlador='PID'):
    """
    Ejecuta el control de temperatura con el tipo de controlador seleccionado
    tipo_controlador: 'PD', 'PI', o 'PID'
    """
    lab = tclab.TCLab()
    
    # Parámetros de control
    Kc = 62.4  # Ganancia proporcional
    tauI = 18.9295  # Tiempo integral
    tauD = 4.7324  # Tiempo derivativo
    Q_bias = 0.0  # Bias
    ierr = 0.0  # Error integral acumulado
    prev_err = 0.0  # Error anterior (para término derivativo)
    
    # Inicializar listas
    n = 600  # Número de muestras
    T1 = [0.0] * n  # Temperatura en sensor 1
    T2 = [0.0] * n  # Temperatura en sensor 2
    Q1 = [0.0] * n  # Señal de control para calentador 1
    SP1 = [40.0] * n  # Setpoint (temperatura deseada)
    
    # Para almacenar los componentes del controlador
    P_component = [0.0] * n  # Componente proporcional
    I_component = [0.0] * n  # Componente integral
    D_component = [0.0] * n  # Componente derivativo
    
    # Configurar la gráfica en modo interactivo
    plt.ion()
    
    try:
        print(f"Iniciando control {tipo_controlador}...")
        for i in range(n):
            # Leer temperatura actual
            T1[i] = lab.T2
            T2[i] = lab.T1
            
            # Cálculo del error
            err = SP1[i] - T1[i]
            
            # Componente proporcional - siempre se usa
            P_component[i] = Kc * err
            
            # Componente integral - solo para PI y PID
            if tipo_controlador in ['PI', 'PID']:
                ierr += err
                I_component[i] = (Kc / tauI) * ierr
            
            # Componente derivativo - para PD y PID
            if tipo_controlador in ['PD', 'PID']:
                deriv = err - prev_err
                D_component[i] = Kc * tauD * deriv
                prev_err = err
            
            # Calcular la acción de control según el tipo de controlador
            if tipo_controlador == 'PD':
                Q1[i] = Q_bias + P_component[i] + D_component[i]
            elif tipo_controlador == 'PI':
                Q1[i] = Q_bias + P_component[i] + I_component[i]
            else:  # PID
                Q1[i] = Q_bias + P_component[i] + I_component[i] + D_component[i]
            
            # Anti-windup
            if Q1[i] >= 100:
                Q1[i] = 100
                if tipo_controlador in ['PI', 'PID']:
                    ierr -= err  # Corregir el error integral acumulado
            elif Q1[i] <= 0:
                Q1[i] = 0
                if tipo_controlador in ['PI', 'PID']:
                    ierr -= err  # Corregir el error integral acumulado
            
            # Aplicar señal de control
            if i > 10:
                lab.Q1(Q1[i])
            else:
                Q1[i] = 0.0
                lab.Q1(0)
            
            # Para actualizar cada 10 segundos pero seguir muestreando cada segundo:
            # 1. Solo se actualiza la gráfica cada 10 iteraciones
            # 2. Pero se continua tomando muestras y aplicando control cada iteración
            
            if i % 10 == 0 or i == n-1:  # Actualizar gráfica cada 10 segundos o en la última iteración
                # Gráfica de temperatura
                plt.clf()
                plt.subplot(2, 1, 1)
                plt.plot(T1[:i+1], 'r-o', label='T1')
                plt.plot(T2[:i+1], 'b-o', label='T2')
                plt.plot(SP1[:i+1], 'k--', label='SP')
                plt.ylabel('Temperatura (°C)')
                plt.title(f'Control {tipo_controlador} de Temperatura')
                plt.grid(True)
                plt.legend()
                
                # Gráfica de PWM (Q1)
                plt.subplot(2, 1, 2)
                plt.plot(Q1[:i+1], 'b-', label='Q1 (PWM)')
                
                # También graficar los componentes del controlador
                plt.plot(P_component[:i+1], 'g-', label='P')
                
                if tipo_controlador in ['PD', 'PID']:
                    plt.plot(D_component[:i+1], 'c-', label='D')
                    
                if tipo_controlador in ['PI', 'PID']:
                    plt.plot(I_component[:i+1], 'y-', label='I')
                
                plt.ylabel('PWM (%)')
                plt.title('Señal de control Q1')
                plt.xlabel('Tiempo (s)')
                plt.grid(True)
                plt.legend()
                
                plt.tight_layout()
                plt.pause(0.05)  # Pequeña pausa necesaria para actualizar la gráfica
                print(f"Iteración {i}: T1={T1[i]:.2f}°C, Q1={Q1[i]:.2f}%")
                
            time.sleep(1)  # Esperar un segundo entre muestras
        
    except KeyboardInterrupt:
        # En caso de interrupción, apagar y cerrar conexión
        print("\nDetención por usuario. Apagando calentadores y cerrando conexión.")
    
    finally:
        # Asegurar que siempre se apaguen los calentadores y se cierre la conexión
        lab.Q1(0)
        lab.Q2(0)
        lab.close()
        
        # Guardar resultados
        t = np.arange(n)
        
        # Crear DataFrame con los datos
        data = {
            'Tiempo (s)': t,
            'Setpoint (SP)': SP1,
            'Temperatura (T1)': T1,
            'Control PWM (Q1)': Q1,
            'Componente P': P_component
        }
        
        # Añadir componentes según el controlador usado
        if tipo_controlador in ['PD', 'PID']:
            data['Componente D'] = D_component
        if tipo_controlador in ['PI', 'PID']:
            data['Componente I'] = I_component
        
        df = pd.DataFrame(data)
        
        # Generar timestamp para nombrar los archivos
        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        
        # Guardar datos en CSV
        csv_filename = f'registro_{tipo_controlador}_TempLab_{timestamp}.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Datos guardados en '{csv_filename}'")
        
        # Crear gráfica final para guardar como imagen
        plt.figure(figsize=(12, 8))
        
        # Gráfica 1: Temperatura vs Tiempo
        plt.subplot(3, 1, 1)
        plt.plot(t, T1, 'r-', label='T1')
        plt.plot(t, SP1, 'k--', label='SP')
        plt.ylabel('Temperatura (°C)')
        plt.title(f'Control {tipo_controlador} de Temperatura')
        plt.grid(True)
        plt.legend()
        
        # Gráfica 2: Señal de control (PWM) vs Tiempo
        plt.subplot(3, 1, 2)
        plt.plot(t, Q1, 'b-', label='Q1 (PWM)')
        plt.ylabel('PWM (%)')
        plt.title('Señal de control Q1')
        plt.grid(True)
        plt.legend()
        
        # Gráfica 3: Componentes del controlador vs Tiempo
        plt.subplot(3, 1, 3)
        plt.plot(t, P_component, 'g-', label='P')
        if tipo_controlador in ['PD', 'PID']:
            plt.plot(t, D_component, 'c-', label='D')
        if tipo_controlador in ['PI', 'PID']:
            plt.plot(t, I_component, 'y-', label='I')
        plt.ylabel('Magnitud de componentes (%)')
        plt.xlabel('Tiempo (s)')
        plt.title('Componentes del controlador')
        plt.grid(True)
        plt.legend()
        
        # Guardar imagen
        image_filename = f'grafico_{tipo_controlador}_TempLab_{timestamp}.png'
        plt.tight_layout()
        plt.savefig(image_filename)
        print(f"Gráfica guardada en '{image_filename}'")
        
        plt.ioff()  # Desactivar modo interactivo
        plt.show()  # Mostrar la gráfica final
        
        return df  # Retornar el dataframe con los resultados

# Ejemplo de uso:
if __name__ == "__main__":
    print("Selecciona el tipo de controlador:")
    print("1 - Control Proporcional-Derivativo (PD)")
    print("2 - Control Proporcional-Integral (PI)")
    print("3 - Control Proporcional-Integral-Derivativo (PID)")
    
    opcion = input("Ingresa el número de la opción deseada (1, 2 o 3): ")
    
    if opcion == '1':
        ejecutar_control('PD')
    elif opcion == '2':
        ejecutar_control('PI')
    elif opcion == '3':
        ejecutar_control('PID')
    else:
        print("Opción no válida. Ejecutando PID por defecto.")
        ejecutar_control('PID')

## Etapa 4 Validacion y Analisis


### Respuesta ITAE
![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/Resultados%20Q1/grafico_PID_TempLab_2025-05-15_17-34-50.png?raw=true)

### Respuesta Zigler Nichols
![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/Resultados%20Q2/grafico_PID_TempLab_2025-05-15_16-44-19.png?raw=true)






### Control selectivo de las componentes

#### Controlador PD: se aplican las componentes proporcional y derivativa
1. Respuesta del sistema (gráfico superior):

    La señal de temperatura $T_1$ responde rápidamente al cambio de consigna gracias a la acción derivativa, que atenúa oscilaciones anticipando la tendencia del error. Sin embargo, se nota claramente que la temperatura nunca llega exactamente al setpoint, lo que confirma la presencia de error en estado estacionario, típico de un sistema sin componente integral.

2. Señal de control PWM (gráfico central):

    Se observa un comportamiento altamente pulsante (PWM al 100% o 0%), lo que sugiere que el controlador está continuamente reaccionando a pequeñas variaciones. Estos pulsos se deben a la alta ganancia derivativa, que es muy sensible al ruido o pequeñas fluctuaciones en la señal de temperatura.

3. Componentes del controlador (gráfico inferior):

    El componente derivativo muestra actividad intensa al inicio de cada ciclo y luego disminuye, reacciona fuertemente al cambio brusco de error, pero luego se reduce al estabilizarse la pendiente del error. El componente proporcional se mantiene más estable, aportando una base constante de corrección.  \\

El controlador PD mejora el tiempo de respuesta y atenúa el sobreimpulso. Pero, no puede eliminar el error en estado estacionario. Esto es crítico en aplicaciones donde se requiere alcanzar y mantener el setpoint con precisión, como en control térmico de procesos.

![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/PD_40.png?raw=true)

#### Controlador PI: se aplican las componentes proporcional e integral
1. Respuesta del sistema (gráfico superior):
    El sistema logra llegar y mantenerse en el setpoint (~40 °C), lo que es gracias a la acción integral, que elimina el error en estado estacionario, aunque la curva de temperatura muestra un sobreimpulso moderado al inicio hay una estabilización adecuada. Hay algo de ruido o pequeñas oscilaciones en la temperatura después de estabilizar.

2. Señal de control Q1 (PWM) (gráfico central):
    El PWM empieza con una señal de 100%, Luego, el comportamiento es mucho más modulante y oscilatorio que en el controlador PD. Eso es típico del PI, donde la acción integral continúa empujando hasta compensar cualquier error acumulado. La señal se mantiene en valores entre 0 % y ~70 % durante la regulación, mostrando que el sistema ya no necesita actuar al 100 % para mantener el setpoint.

3. Componentes del controlador (gráfico inferior):
    El componente Proporcional responde al error actual dismunuyendo rápidamente al acercarse al setpoint, mientras la componente integral se acumula con el tiempo y sostiene la acción de control, incluso cuando el error ya es pequeño. Esto es lo que permite eliminar el error en estado estacionario. Ambas señales convergen a un valor estable y controlado.

![image.png](https://github.com/jbcgames/Lab_5_Control/blob/main/PI_40.png?raw=true)



### Script de analisis de datos:

In [16]:
import pandas as pd

def analizar_respuesta_sistema(nombre_archivo_csv, tolerancia=0.05):
    # Leer el archivo
    df = pd.read_csv(nombre_archivo_csv)
    
    tiempo = df["Tiempo (s)"]
    sp = df["Setpoint (SP)"]
    temp = df["Temperatura (T)"]
    
    setpoint = sp.iloc[0]  # asumimos que es constante

    # Calcular sobreimpulso
    temp_max = temp.max()
    overshoot = max(0, ((temp_max - setpoint) / setpoint) * 100)  # en %

    # Calcular error de estado estacionario (últimos 50 valores)
    e_ss = abs(setpoint - temp.tail(50).mean())

    # Calcular tiempo de asentamiento (dentro del ±5% del setpoint)
    margen_superior = setpoint * (1 + tolerancia)
    margen_inferior = setpoint * (1 - tolerancia)

    dentro_margen = (temp >= margen_inferior) & (temp <= margen_superior)
    for i in range(len(temp) - 1, -1, -1):
        if not dentro_margen.iloc[i]:
            settling_time = tiempo.iloc[i + 1] if (i + 1) < len(tiempo) else tiempo.iloc[-1]
            break
    else:
        settling_time = tiempo.iloc[0]  # si nunca sale de la banda

    # Resultados
    
    print(f"Sobreimpulso (%): {overshoot:.2f}")
    print(f"Tiempo de asentamiento (s): {settling_time:.2f}")
    print(f"Error en estado estacionario (°C): {e_ss:.2f}")
    
print("Resultados de la respuesta del sistema con ITAE:")
analizar_respuesta_sistema('https://raw.githubusercontent.com/jbcgames/Lab_5_Control/main/Resultados%20Q1/registro_PID_TempLab_2025-05-15_17-34-50.csv')
print(" ")
print("Resultados de la respuesta del sistema con Ziegler-Nichols:")
analizar_respuesta_sistema('https://raw.githubusercontent.com/jbcgames/Lab_5_Control/main/Resultados%20Q2/registro_PID_TempLab_2025-05-15_16-44-19.csv')

Resultados de la respuesta del sistema con ITAE:
Sobreimpulso (%): 2.07
Tiempo de asentamiento (s): 153.00
Error en estado estacionario (°C): 0.31
 
Resultados de la respuesta del sistema con Ziegler-Nichols:
Sobreimpulso (%): 1.58
Tiempo de asentamiento (s): 171.00
Error en estado estacionario (°C): 0.06
